# Обработка данных заемщиков для кредитного отдела банка

<b><font size=5>Содержание</font></b><a name="to_content."></a>
* [0. Описание проекта](#0.)
* [1. Изучение общей информации](#1.)
* [2. Предобработка данных](#2.)
     - [2.1 Обработка пропусков](#2.1)
     - [2.2 Замена типа данных](#2.2)
     - [2.3 Обработка дубликатов](#2.3)
     - [2.4 Лемматизация](#2.4)
     - [2.5 Категоризация данных](#2.5)
* [3. Поиск кореляций](#3.)
     - [3.1 Проверка связи между наличием детей и возвратом кредита в срок](#3.1)
     - [3.2 Проверка связи между семейным положением и возвратом кредита в срок](#3.2)
     - [3.3 Проверка связи между уровнем дохода и возвратом кредита в срок](#3.3)
     - [3.4 Проверка связи между целью займа и возвратом кредита в срок](#3.4)
* [4. Общий вывод](#4.)

<b><font size=5>Описание проекта</font></b><a name="0."></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

**Исследование надёжности заёмщиков**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<b><font size=5>Изучение общей информации</font></b><a name="1."></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [1]:
import pandas as pd

data=pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
#сделаем выборку из 10 записей, чтобы рассмотреть датасет подробнее
data.sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7979,0,-180.279704,49,Высшее,0,Не женат / не замужем,4,F,сотрудник,0,170693.563511,заняться высшим образованием
8787,0,-1496.436457,22,среднее,1,Не женат / не замужем,4,F,сотрудник,0,85699.479290,сделка с автомобилем
2706,1,NaN,40,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка недвижимости
14526,0,-1882.932787,34,среднее,1,Не женат / не замужем,4,M,сотрудник,0,232087.543238,образование
10667,1,-777.693677,27,среднее,1,женат / замужем,0,M,сотрудник,0,145920.280935,покупка коммерческой недвижимости
12583,0,-4725.401998,56,среднее,1,в разводе,3,F,компаньон,0,239253.735723,получение высшего образования
15733,0,-1249.392484,46,высшее,0,гражданский брак,1,M,компаньон,0,196234.792132,операции с недвижимостью
20411,1,-133.883739,43,среднее,1,женат / замужем,0,F,компаньон,0,61442.128025,строительство недвижимости
11621,0,-513.734323,57,высшее,0,женат / замужем,0,M,сотрудник,0,258177.093962,покупка коммерческой недвижимости
4770,1,NaN,40,среднее,1,Не женат / не замужем,4,F,компаньон,0,NaN,получение дополнительного образования


**Выводы**

 - Датасет содержит 12 столбцов и 2125 строк, в столбцах days_employed и total_income есть пропущенные значения.
 - Представленные следующие типы данных: float64(2 столбца: days_employed и total_income), int64(5 столбцов: children, dob_years, education_id, family_status_id и debt), object(5 столбцов:education, family_status, gender, income_type, purpose)
 - Уже видны нереальные и отрицательные значения в столбце с трудовым стажем

<b><font size=5>Предобработка данных</font></b><a name="2."></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

<b><font size=4>Обработка пропусков</font></b><a name="2.1"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#отфильтруем датасет по пропускам, чтобы изучить их
data.loc[data['days_employed'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


**Вывод:**
похоже, что отсутствующие данные в столбцах days_employed и total_income находятся на тех же строках. Проверим это

In [5]:
#отфильтруем только строки, где пропущены значения ИЛИ в days_employed ИЛИ в total_income. 
#Если их окажется 21525-19351=2174,то значит пропущенные данные по доходам и стажу находятся в тех же строках.
data.loc[(data['days_employed'].isnull()) | (data['total_income'].isnull())].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


**Вывод:** пропущенные значения по столбцам days_employed и в total_income находятся в одной строке. Связей с другими столбцами нет.

- Отсутсвующие данные - 2174/21525 = 10% от выборки. Просто избавиться от строк не получится. 

- В данной ситуации считаю самым разумным заполнить пропуски total_income медианными значениями дохода по виду дохода (income_type)

- Пропуски в days_employed правильно было бы заполнить средним стажем по возрастной группе (для начала разделив датасет по возрастным группам) или даже с использованием стохастической линейной регрессии (судя по статье после одного из уроков), но т.к. вопросов относительно связи между стажем работы и возвратом в срок не задается, я заполню эти пропуски средними значениями

- Так как входные данные - статистика платежеспособности клиентов, то доходы и стаж являются важными метриками в такой статистике. Трудно представить, что банк дает возможность не указывать эту информацию заемщикам. Я бы предположил, что это ошибка выгрузки и попробовал связаться с человеком, который выгружал данные.

In [6]:
#проверим уникальные значения колонок типа 'object' на наличие NaN и None в текстовом формате
def unique_values(df):
    for column in df:
        if df[column].dtype == 'object':
            unique_list = df[column].unique()
            print(unique_list)
    
unique_values(data)

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образо

**Вывод**

- Пропущенных значений текстового типа 'Nan', 'None' не найдено
- В столбце образование имеются дубликаты с разным регистром
- XNA в столбце с полом, вероятно выставлено намерено, заменять не буду

In [7]:
#проверим максимальные и минимальные величины в численных столбцах
from pandas.api.types import is_numeric_dtype
def min_max_values(df):
    for column in df:
        if is_numeric_dtype(df[column]):
            max = df[column].max()
            min = df[column].min()
            print(column, ':', min, max)
            
min_max_values(data)

children : -1 20
days_employed : -18388.949900568383 401755.40047533
dob_years : 0 75
education_id : 0 4
family_status_id : 0 4
debt : 0 1
total_income : 20667.26379327158 2265604.028722744


**Вывод** 

Необходимо обработать следующие столбцы:
- Дети (-1 и 20). Вероятно всего опечатки ручного ввода, под которыми кроются 1 и 2 ребенка. Я бы уточнил, как вводятся данные о детях в датасет, чтобы убедиться в этом. Эти строки (113) составляют 1-2% от всего числа строк заемщиков с детьми и еще меньше от общего числа строк. Сейчас их можно отбросить
- Возраст (0). Вероятно вычисляется с ошибками по причине некорректного заполнения даты рождения, нужно сообщить о проблеме
- Трудовой стаж(отрицательные значения, плюс невероятные значения 401755/365=1100 лет). 

In [8]:
#до обработки данных проверим построчно дубликаты
data.loc[data.duplicated()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 2849 to 21415
Data columns (total 12 columns):
children            54 non-null int64
days_employed       0 non-null float64
dob_years           54 non-null int64
education           54 non-null object
education_id        54 non-null int64
family_status       54 non-null object
family_status_id    54 non-null int64
gender              54 non-null object
income_type         54 non-null object
debt                54 non-null int64
total_income        0 non-null float64
purpose             54 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 5.5+ KB


**Вывод:** всего 54 теоретических дубликата с NaN в стаже и доходе. Я думаю, что это может быть просто совпадение, т.к. единственный столбец, который реально отличает эти строки "цель займа", но и среди целей уникальных формулировок у нас всего штук 30. Дети, пол, образование могут легко совпасть

In [9]:
#вычисли медианный доход по типу дохода
median_income = data.groupby('income_type')['total_income']\
    .median()\
    .reset_index()\
    .rename(columns={'total_income': 'median_income'})\
    .round(2)
median_income

,income_type,median_income
0,безработный,131339.75
1,в декрете,53829.13
2,госслужащий,150447.94
3,компаньон,172357.95
4,пенсионер,118514.49
5,предприниматель,499163.14
6,сотрудник,142594.40
7,студент,98201.63


In [10]:
#добавим медианный доход по типу дохода и заполним пропуски
data = data.merge(median_income, how='left', on='income_type')
data.loc[data['total_income'].isna(), 'total_income'] = data['median_income']

In [11]:
#проверим info()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
median_income       21525 non-null float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.9+ MB


In [12]:
#не изменились ли медианы
data.groupby('income_type')['total_income']\
    .median()\
    .reset_index()\
    .rename(columns={'total_income': 'median_income'})\
    .round(2)

,income_type,median_income
0,безработный,131339.75
1,в декрете,53829.13
2,госслужащий,150447.94
3,компаньон,172357.95
4,пенсионер,118514.49
5,предприниматель,499163.14
6,сотрудник,142594.40
7,студент,98201.63


**Вывод**

- NaN в total_income были заполнены медианами по типу дохода (income_type) корректно 

In [2]:
#бывшие NaN
display(data.loc[82].to_frame(), data.loc[65].to_frame())

NameError: name 'data' is not defined

<hr style="border: 1px solid #000;"> 

In [15]:
#посмотрим, сколько у нас нулей в столбце с возрастами
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [16]:
#вычислим средний возраст и заполним им нулевые значения
dob_years_avg = data['dob_years'].mean()
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_avg
data.loc[data['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income


**Вывод:** заменили пропуски в стаже на сердние значения, заменили нули в возрасте на средние значения. Нулей было 101, картину данных не исказим

In [17]:
#умножим на -1 все отрицательные величины в столбце days_employed
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed'] * (-1)
data.loc[data['days_employed'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income


In [18]:
#сделаем категоризацию заемщиков по возрастам и найдем медианный стаж, чтобы заполнить пропуски
#медиану используем по причине выбросов в значениях стажа
def age_group(age):
    if age <= 30:
            return '18-30'
    if age <= 40:
            return '30-40'
    if age <= 50:
            return '40-50'
    if age <= 60:
            return '50-60'
    return '60+'

In [19]:
data['age_group'] = data['dob_years'].apply(age_group)

In [20]:
median_days_employed = data.groupby('age_group')['days_employed']\
    .median()\
    .reset_index()\
    .rename(columns={'days_employed': 'median_days_employed'})\
    .round(2)
median_days_employed

,age_group,median_days_employed
0,18-30,1046.10
1,30-40,1630.19
2,40-50,2199.73
3,50-60,6481.04
4,60+,356191.14


**Вывод:** В пояснении к данным указано, что это общий трудовой стаж, а не на текущей должности/текущем месте. Медианы на мой взгляд занижают рабочий стаж, не может быть у половины 30-40 летних людей стаж работы менее 5 лет. Кроме группы 60+, где значения нереальные.

In [21]:
#попробуем средние
mean_days_employed = data.groupby('age_group')['days_employed']\
    .mean()\
    .reset_index()\
    .rename(columns={'days_employed': 'mean_days_employed'})\
    .round(2)
mean_days_employed

,age_group,mean_days_employed
0,18-30,2027.15
1,30-40,4691.12
2,40-50,17693.27
3,50-60,153165.11
4,60+,290708.55


**Вывод:** средние значительно завышают трудовой стаж. Для групп 18-30 и 30-40 лет значения похожи на адекватные, но дальше начинаются неадекватные значения

In [22]:
#вычислим среднее значение трудового стажа и заполним им пропуски просто в качестве упражнения
#т.к. эти данные нам не пригодятся в текущем анализе
age_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(age_employed_avg)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
median_income       21525 non-null float64
age_group           21525 non-null object
dtypes: float64(4), int64(4), object(6)
memory usage: 3.1+ MB


In [23]:
#проверим в скольких строках не соблюдается условие годы стажа < годы жизни
data.loc[(data['days_employed'] / 365) > data['dob_years']]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.49,50-60
12,0,66914.728907,65.0,среднее,1,гражданский брак,1,M,пенсионер,0,118514.490000,сыграть свадьбу,118514.49,60+
18,0,400281.136913,53.0,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,118514.49,50-60
24,1,338551.952911,57.0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,118514.49,50-60
25,0,363548.489348,67.0,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,118514.49,60+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21508,0,386497.714078,62.0,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,118514.49,60+
21509,0,362161.054124,59.0,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,118514.49,50-60
21510,2,66914.728907,28.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.400000,приобретение автомобиля,142594.40,18-30
21518,0,373995.710838,59.0,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем,118514.49,50-60


**Вывод**: отрицательные значения заменены, но это не сильно помогло, т.к в 5619 строк (26%) указаны нереальные значения. К сожалению отбросить мы их не можем, но к счастью в анализе они нам не пригодятся. Необходимо будет обязательно указать на это коллегам

In [24]:
#отбросим строки со значением 'children' -1 и 20
data = data.drop(data[data['children'] < 0].index)
data = data.drop(data[data['children'] == 20].index)
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

<b><font size=4>Замена типа данных</font></b><a name="2.2"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [25]:
#заменим Float64 на Int64 в столбцах days_employed, dob_years, median_income и total_income
data[['days_employed', 'total_income', 'dob_years', 'median_income']] = \
data[['days_employed', 'total_income', 'dob_years', 'median_income']].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 14 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
median_income       21402 non-null int64
age_group           21402 non-null object
dtypes: int64(8), object(6)
memory usage: 2.4+ MB


In [26]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594,40-50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594,30-40
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594,30-40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594,30-40
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514,50-60
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,172357,18-30
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,172357,40-50
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,142594,40-50
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,142594,30-40
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,142594,40-50


**Выводы**

- Данные приведены к типу int с помощью .astype, т.к. столбцы уже представлены в числовом формате, но вещественном числовом

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 14 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
median_income       21402 non-null int64
age_group           21402 non-null object
dtypes: int64(8), object(6)
memory usage: 2.4+ MB


In [28]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594,40-50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594,30-40
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594,30-40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594,30-40
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514,50-60
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,172357,18-30
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,172357,40-50
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,142594,40-50
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,142594,30-40
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,142594,40-50


<b><font size=4>Обработка дубликатов</font></b><a name="2.3"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [29]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод:** 
- Избежали дублирования уровня образования с учетом регистра методом str.lower(). До этого значения были заполнены с заглавной буквы или полностью верхним регистром, вероятно ошибки ручного ввода
- Среди целей займов есть дубликаты по значению, разобраться с этим поможет группировка по леммам
- Поиск строк-дубликатов затруднен по причине отсутствия какого-либо ключа заемщика или id

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 14 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
median_income       21402 non-null int64
age_group           21402 non-null object
dtypes: int64(8), object(6)
memory usage: 2.4+ MB


In [31]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,142594,40-50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,142594,30-40
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,142594,30-40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,142594,30-40
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,118514,50-60
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,172357,18-30
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,172357,40-50
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,142594,40-50
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,142594,30-40
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,142594,40-50


In [32]:
#поиск дубликатов построчно
data.loc[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
2849,0,66914,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи,142594,40-50
3290,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,118514,50-60
4182,1,66914,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба,142594,30-40
4851,0,66914,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,118514,50-60
5557,0,66914,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,118514,50-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,66914,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование,118514,60+
21032,0,66914,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием,118514,50-60
21132,0,66914,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью,142594,40-50
21281,1,66914,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости,142594,18-30


<b><font size=4>Лемматизация</font></b><a name="2.4"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [33]:
#напишем простую лямбда функцию для лемматизации столбца 'purpose', чтобы в результате он содержал листы с леммами.
#используем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()
lemmas = lambda x: m.lemmatize(str(x))
data['purpose'] = data['purpose'].apply(lemmas)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",142594,40-50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",142594,30-40
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",142594,30-40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",142594,30-40
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",118514,50-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,"[операция, , с, , жилье, \n]",172357,40-50
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,"[сделка, , с, , автомобиль, \n]",118514,60+
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,"[недвижимость, \n]",142594,30-40
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,"[на, , покупка, , свой, , автомобиль, \n]",142594,30-40


In [34]:
#теперь сосчитаем леммы, чтобы найти самые популярные для категоризации
#используем метод most_common у Counter из пайтон модуля collections
from collections import Counter
all_lemmas = data['purpose'].apply(pd.Series).stack().reset_index(drop = True)
Counter(all_lemmas).most_common(15)

[(' ', 33485),
 ('\n', 21402),
 ('недвижимость', 6330),
 ('покупка', 5880),
 ('жилье', 4450),
 ('автомобиль', 4288),
 ('образование', 3997),
 ('с', 2906),
 ('операция', 2593),
 ('свадьба', 2337),
 ('свой', 2224),
 ('на', 2218),
 ('строительство', 1870),
 ('высокий', 1368),
 ('получение', 1311)]

**Вывод:** с помощью лемматизации можно выделить главные категории займов: недвижимость(включая коммерческую, некоммерческую, строительство этой недвижимости и т.д. - при необходимости позже можно будет разделить на коммерческую и некоммерческую), автомобиль, образование, лечение, свадьба и соответсвенно "иное"

<b><font size=4>Категоризация данных</font></b><a name="2.5"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

Для ответа на вопросы необходимо классифицировать данные по категориям займов, наличию детей, семейному положению и уровню дохода:

- По категориям займов будем делить на следующие категории: недвижимость(включая коммерческую, некоммерческую, частную, строительство этой недвижимости и т.д. - при необходимости позже можно будет разделить на коммерческую и некоммерческую), автомобиль, образование, операция, свадьба и соответсвенно "иное"

- По наличию детей: очевидно из названия, что это будет булев столбец. Тем не менее, нужно будет посмотреть связь между количеством детей и возвратом кредита в срок
- По семейному положению: каждое из этих положений уникально по своему в финансовом плане, я бы не сжимал эти категории
- По уровню дохода: разделить выборку по доходам на квартили и использовать каждый квартель как отдельную группу

In [35]:
#напишем три функции для категоризации данных по категориям, описанным выше
def purpose_group(purpose):
    if ('недвижимость' in purpose or 'жилье' in purpose):
            return 'недвижимость'
    if 'автомобиль' in purpose:
            return 'автомобиль'
    if 'лечение' in purpose:
            return 'лечение'
    if 'образование' in purpose:
            return 'образование'
    if 'свадьба' in purpose:
            return 'свадьба'
    return 'иное'

In [36]:
#проверим функцию
print(purpose_group(['недвижимость', 'жилье']))
print(purpose_group(['автомобиль']))
print(purpose_group(['лечение']))
print(purpose_group(['образование']))
print(purpose_group(['свадьба']))
print(purpose_group(['тусы на Бали']))

недвижимость
автомобиль
лечение
образование
свадьба
иное


In [37]:
#напишем следующую функцию для категоризации и проверим
has_kids = lambda x: x>0

In [38]:
print(has_kids(2))
print(has_kids(0))

True
False


In [39]:
#функция категоризации по доходам
high = data['total_income'].quantile(.75)
medium_high = data['total_income'].quantile(.50)
medium_low = data['total_income'].quantile(.25)
def income_level(income):
    if income > high:
            return 'высокий'
    if income > medium_high:
            return 'выше среднего'
    if income > medium_low:
            return 'ниже среднего'
    return 'низкий'

In [40]:
#создадим новые столбцы с категориями
data['purpose_group'] = data['purpose'].apply(purpose_group)
data['has_kids'] = data['children'].apply(has_kids)
data['income_level'] = data['total_income'].apply(income_level)

In [41]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median_income,age_group,purpose_group,has_kids,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",142594,40-50,недвижимость,True,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",142594,30-40,автомобиль,True,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",142594,30-40,недвижимость,False,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",142594,30-40,образование,True,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",118514,50-60,свадьба,False,выше среднего
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,"[покупка, , жилье, \n]",172357,18-30,недвижимость,False,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,"[операция, , с, , жилье, \n]",172357,40-50,недвижимость,False,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,"[образование, \n]",142594,40-50,образование,False,ниже среднего
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,"[на, , проведение, , свадьба, \n]",142594,30-40,свадьба,True,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,"[покупка, , жилье, , для, , семья, \n]",142594,40-50,недвижимость,False,выше среднего


**Вывод:** выполнили категоризацию данных на основании критериев, перечисленных в начале этого подраздела

<b><font size=5>Поиск корреляций</font></b><a name="3."></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

<b><font size=4>Проверка связи между наличием детей и возвратом кредита в срок</font></b><a name="3.1"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [42]:
data['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [43]:
#благодаря тому, что True и False это 1 и 0, мы можем использовать фукцию mean к столбцу debt
#чтобы посчитать процент людей с задолжностями
data_pivot = data.pivot_table \
(index=['has_kids'], values='debt', aggfunc='mean').round(3).reset_index()
data_pivot

,has_kids,debt
0,False,0.075
1,True,0.092


In [44]:
data_pivot = data.pivot_table \
(index=['children'], values='debt', aggfunc='mean').round(3).reset_index()
data_pivot

,children,debt
0,0,0.075
1,1,0.092
2,2,0.094
3,3,0.082
4,4,0.098
5,5,0.000


**Вывод:** клиентов с детьми имевших задолжности по кредитам 9.2% от выборки, клиентов без детей имевших задолжность 7.5% от выборки. Получается, клиенты с детьми примерно на |7.5-9.2|/7.5=23% чаще имеют в истории задолжность по кредиту. В то же время сложно определить корреляцию между количеством детей и задержкой платежей по причине низкого количества записей с children > 2

<b><font size=4>Проверка связи между семейным положением и возвратом кредита в срок</font></b><a name="3.2"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [45]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [46]:
data.groupby('family_status')['debt'].mean().round(3)

family_status
Не женат / не замужем    0.098
в разводе                0.071
вдовец / вдова           0.066
гражданский брак         0.093
женат / замужем          0.075
Name: debt, dtype: float64

**Вывод:** получается, что люди, которые не состоят и ранее не состояли в официальном браке (т.е не женатые/замужние или состоящие в неофициальном), чаще имеют в истории задолжность по кредиту. Примем женатых людей за базовую линию (разведеных и вдовцов мало для достоверности), тогда люди в гражданском браке на (7.5-9.3)/7.5=24% чаще имеют в истории задолжность по кредиту, а не женатые/замужние на 31% чаще 

<b><font size=4>Проверка связи между уровнем дохода и возвратом кредита в срок</font></b><a name="3.3"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [47]:
data.groupby('income_level')['debt'].mean().round(3)

income_level
высокий          0.072
выше среднего    0.085
ниже среднего    0.087
низкий           0.080
Name: debt, dtype: float64

**Вывод:** люди с высоким доходом на 10% реже 
    имеют в истории задолжность по кредиту, в то время как люди с 
    доходами между .25 и .5 квантилями примерно на 7.5% чаще по сравнению с низкими доходами

<b><font size=4>Проверка связи между целью займа и возвратом кредита в срок</font></b><a name="3.4"></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

In [48]:
data.groupby('purpose_group')['debt'].mean().round(3)

purpose_group
автомобиль      0.093
недвижимость    0.072
образование     0.092
свадьба         0.078
Name: debt, dtype: float64

**Вывод:** если принять категорию "недвижимость" как базовую линию, то платежи по кредитам за свадьбу задерживают на 8.3% чаще, а за образование и автомобили на 28% чаще

<b><font size=5>Общий вывод</font></b><a name="4."></a><br/>
[<font size="2">(к содержанию)</font>](#to_content.)

Ответив на вопросы из шага три можно сделать следующие выводы:

- Реже всего меют в истории задолжность по кредиту люди, состоящие в браке, с высоким доходом, без детей с займом на недвижимость
- Наибольшее влияние оказывает цель займа - никто не хочет потерять крышу над головой, и брачный статус  